### Homework 5

* Create a class and name it Z-test this class have 
* Create a class and name it T-test this class includes One-sample, two-sample, and paired t-test
* Create a class and name it ANOVA it includes one way and two ways
* Create a class and name it Chi-Square
* Create a class and name it AB testing 


In [30]:
from scipy import stats
import numpy as np

class Ztest:
        
    def __init__(self, x, sigma, test_value = 0, alpha = 0.05, H1 = 'unequal'):
        ''' 
        Runs a one-sample Z-test based on the specified input
        
        Input:
          x          : the data sample (a vector)
          sigma      : the standard deviation of the population from which x was drawn
          test_value : the hypothesized population mean
          alpha      : significance level for hypothesis testing
          H1         : alternative hypothesis: 'unequal' (default), 'smaller', or 'larger'
        '''
        # set attributes
        self.x = np.array(x)
        self.sigma = sigma
        self.test_value = test_value
        self.alpha = alpha
        self.H1 = H1
        # run the test 
        self.run_test()
        # show results
        self.print_results()
    
    def deviates_from_normal(self):
        '''Uses rules of thumb on skew and kurtosis (West et al., 1996) to determine if the sample deviates from a normal distribution'''
        kurt = stats.kurtosis(self.x)
        skew = stats.skew(self.x)
        return (-7 < kurt < 7) and (-2 < skew < 2)
    
    def run_test(self):
        '''Run the Z-test'''
        self.z = (self.x.mean() - self.test_value) / (self.sigma / np.sqrt(len(self.x)))        
        if self.H1 == 'unequal':
            self.p_value = 2 * stats.norm.sf(abs(self.z))
        elif self.H1 == 'smaller':
            self.p_value = stats.norm.cdf(self.z)
        elif self.H1 == 'larger':
            self.p_value = 1 - stats.norm.cdf(self.z)
        else:
            raise ValueError(f'Invalid alternative hypothesis: {self.H1}')        
        
    def print_results(self):
        '''Print the result of the statistical test'''
        print(f'---------------------------------')
        print(f'Z-test result')
        print(f'---------------------------------')
        print(f'n           = {len(self.x)}')        
        print(f'sample mean = {self.x.mean():.2f}')
        print(f'sigma       = {self.sigma:.2f}')
        print(f'z           = {self.z:.2f}')
        print(f'p           = {self.p_value:.4f}')
        if self.deviates_from_normal():
            print(f'\nNote: the sample seems to come from a non-normal population (skew: {stats.skew(self.x):.1f}; kurtosis: {stats.kurtosis(self.x):.2f}); consider using a non-parametric test instead')
        


In [ ]:
class ttest:
        
    def __init__(self, x, alpha = 0.05, H1 = 'unequal', test_value = 0):
        ''' 
        Runs a one- or two-sample Student t-test based on the specified input
        
        Input:
          x          : the data sample (a vector or a list with two vectors in case of a two-sample test)
          alpha      : significance level for hypothesis testing
          H1         : alternative hypothesis: 'unequal' (default), 'smaller', or 'larger'
          test_value : the hypothesized population mean (in case of a one-sample test)
        '''
        # set attributes
        self.x = np.array(x)
        self.alpha = alpha
        self.H1 = H1
        self.test_value = test_value
        # run the test 
        self.run_test()
        # show results
        self.print_results()
    
    def deviates_from_normal(self):
        '''Uses rules of thumb on skew and kurtosis (West et al., 1996) to determine if the sample deviates from a normal distribution'''
        kurt = stats.kurtosis(self.x)
        skew = stats.skew(self.x)
        return (-7 < kurt < 7) and (-2 < skew < 2)
    
    def run_test(self):
        '''Run the Z-test'''
        self.z = (self.x.mean() - self.test_value) / (self.sigma / np.sqrt(len(self.x)))        
        if self.H1 == 'unequal':
            self.p_value = 2 * stats.norm.sf(abs(self.z))
        elif self.H1 == 'smaller':
            self.p_value = stats.norm.cdf(self.z)
        elif self.H1 == 'larger':
            self.p_value = 1 - stats.norm.cdf(self.z)
        else:
            raise ValueError(f'Invalid alternative hypothesis: {self.H1}')        
        
    def print_results(self):
        '''Print the result of the statistical test'''
        print(f'---------------------------------')
        print(f'Z-test result')
        print(f'---------------------------------')
        print(f'n           = {len(self.x)}')        
        print(f'sample mean = {self.x.mean():.2f}')
        print(f'sigma       = {self.sigma:.2f}')
        print(f'z           = {self.z:.2f}')
        print(f'p           = {self.p_value:.4f}')
        if self.deviates_from_normal():
            print(f'\nNote: the sample seems to come from a non-normal population (skew: {stats.skew(self.x):.1f}; kurtosis: {stats.kurtosis(self.x):.2f}); consider using a non-parametric test instead')
        


In [31]:
x = [4.186, 4.439, 4.781, 4.388, 4.947, 4.853, 4.889, 4.682, 4.428,  4.533, 4.557, 4.761, 4.491, 4.334, 4.83 , 4.268, 4.68 , 4.437, 5.382, 5.111, 5.096, 5.232, 5.033, 5.57 , 4.474, 4.789, 4.725, 4.84 , 4.817, 4.438, 4.754, 4.966, 4.285, 4.482, 4.396, 4.418, 4.514, 5.383, 5.264, 4.309, 5.058, 4.392, 4.788, 4.934, 4.967, 4.554, 4.42 , 5. , 5.126, 5.082, 4.944, 4.658]
ztest = Ztest(x = x, sigma = 10, test_value = 0, alpha = .05, H1 = 'larger')

---------------------------------
Z-test result
---------------------------------
n           = 5
sample mean = 2.40
sigma       = 10.00
z           = 0.54
p           = 0.2958

Note: the sample seems to come from a non-normal population (skew: -0.8; kurtosis: -0.92); consider using a non-parametric test instead


In [23]:
data = [-30, 30]
test_value = 100

_, pvalue = stats.ttest_1samp(data, test_value)

pvalue.round(2)

0.19

In [ ]:
salary_women = [32000, 17000]  # 32k, 17k
salary_men = [42000, 91000] # 42k, 90k

_, pvalue = stats.ttest_ind(salary_women, salary_men)

pvalue.round(2)